**Find Mininmum Distance from a rental property to any point of interest**

In [12]:
# Import necessary libraries
from pyspark.sql import SparkSession, functions as F
import geopandas as gpd
from shapely.geometry import Point

# Create Spark session
spark = (
    SparkSession.builder.appName('Weekly Rental Listings Preprocessing')
    .config("spark.sql.repl.eagerEval.enabled", True)  # display full dataframe in console
    .config("spark.sql.parquet.cacheMetadata", "true") # cache parquet metadata
    .config("spark.sql.session.timeZone", "Etc/UTC") # set timezone to UTC
    # memory configurations - hopefully will reduce crashing
    .config("spark.driver.memory", "4g") # set driver memory
    .config("spark.executor.memory", "4g") # set executor memory
    .getOrCreate()
)

In [13]:
# Load CSV data into Spark DataFrame
sdf = spark.read.option("escape", '"').csv('../data/raw/domain/rental_listings_2025_09.csv', header=True, inferSchema=True)

# Show the first rows full_address column data 

sdf.select("full_address").limit(10)

full_address
14 Federation Lan...
106/609 Victoria ...
Abbotsford VIC 3067
4/2 Princes Stree...
"45 Park St, Abbot..."
302/14 Trenerry C...
DG04/18 Grosvenor...
Y102/125 Turner S...
204/11 Flockhart ...
207 Langridge Str...


In [14]:
sdf.limit(10)
sdf

age_0_to_19,age_20_to_39,age_40_to_59,age_60_plus,agency_name,agent_name,appointment_only,avg_days_on_market,bathrooms,bedrooms,car_spaces,description,family_percentage,features_list,first_listed_date,full_address,image_urls,inspection_text,land_area,last_sold_date,latitude,listing_status,listing_tag,listing_url,long_term_resident,longitude,median_rent_price,median_sold_price,number_of_photos,number_sold,owner_percentage,postcode,property_features,property_id,property_type,rental_price,renter_percentage,schools,single_percentage,state_abbreviation,street,street_number,structured_features,suburb,unit_number,updated_date,url
0.142523363,0.5,0.226635516,0.130841121,RT Edgar - Northside,Lily Passarelli,False,60,NULL,3,2,Just freshly pain...,0.5304348,Split System Air ...,2025-08-26T16:14:...,14 Federation Lan...,https://rimh2.dom...,NULL,NULL,NULL,-37.796893,live,NULL,https://www.domai...,0.4220963,144.9961565,900,1350000,11,34,0.5895954,3067,"3, ,2, ,2,",17732837,Townhouse,$880 per week,0.410404623,[('Clifton Hill P...,0.4695652,vic,Federation Lane,14,"[{'name': 'Bath',...",Abbotsford,NULL,2025-09-03 10:12:...,https://www.domai...
0.08416759,0.6407119,0.200222462,0.0748980343,Fletchers Project...,Joyee Jiang,False,69,NULL,2,2,Perfectly positio...,0.496447623,NULL,2025-09-02T17:45:...,106/609 Victoria ...,https://rimh2.dom...,NULL,NULL,NULL,-37.8110653,live,NULL,https://www.domai...,0.44155845,145.0076834,660,580000,14,187,0.2623574,3067,"2, ,2, ,1,",17744154,Apartment / Unit ...,$750 per week,0.7376426,[('Yarra Primary ...,0.5035524,vic,Victoria Street,609,"[{'name': 'Gas', ...",Abbotsford,106,2025-09-02 17:45:...,https://www.domai...
0.123569794,0.3340961,0.23569794,0.306636155,RentBetter,NULL,True,85,NULL,1,1,PROPERTY ID: 3999...,0.40625,"Air conditioning,...",2025-09-06T22:53:...,Abbotsford VIC 3067,https://rimh2.dom...,NULL,NULL,NULL,-37.80210950000001,live,NULL,https://www.domai...,0.491228074,145.0019064,510,393000,15,141,0.4144737,3067,"1, ,1, ,1,",17750349,Apartment / Unit ...,$630 per week,0.5855263,[('Sophia Mundi S...,0.59375,vic,NULL,NULL,[{'name': 'Intern...,Abbotsford,NULL,2025-09-06 22:53:...,https://www.domai...
0.174757287,0.5080906,0.220064729,0.09708738,Miles Real Estate...,Carly Tomat,False,32,NULL,2,1,FIRST INSPECTIONS...,0.44921875,"Air conditioning,...",2025-09-01T10:37:...,4/2 Princes Stree...,https://rimh2.dom...,NULL,NULL,NULL,-37.8092053,recentlyUpdated,NULL,https://www.domai...,0.558935344,144.999856,725,1100000,7,42,0.495575249,3067,"2, ,1, ,1,",17739910,Townhouse,$650 per week,0.504424751,[('Abbotsford Pri...,0.55078125,vic,Princes Street,2,[{'name': 'Intern...,Abbotsford,4,2025-09-11 17:00:...,https://www.domai...
0.170111284,0.400635928,0.275039762,0.154213041,Jellis Craig Rich...,Nathan Roberts,False,32,NULL,2,1,Discover this spa...,0.43849206,"Air conditioning,...",2025-09-08T12:31:...,"45 Park St, Abbot...",https://rimh2.dom...,NULL,NULL,NULL,-37.8080424,new,NULL,https://www.domai...,0.4714829,144.9939399,725,1100000,11,42,0.607142866,3067,"2, ,1, ,−,",17751219,House,$750 per week,0.392857134,[('Abbotsford Pri...,0.56150794,vic,Park St,45,"[{'name': 'Gas', ...",Abbotsford,NULL,2025-09-11 09:25:...,https://www.domai...
0.06559406,0.6311881,0.225247532,0.0779702961,Dingle Partners,Shane Dangen,True,69,NULL,2,1,Be first to inspe...,0.5532234,Furnished,2025-07-28T14:38:...,302/14 Trenerry C...,https://rimh2.dom...,NULL,NULL,NULL,-37.7996262,live,NULL,https://www.domai...,0.200892851,145.0020093,660,580000,14,187,0.274939179,3067,"2, ,1, ,1,",17688466,Apartment / Unit ...,$825 per week,0.7250608,[('Sophia Mundi S...,0.4467766,vic,Trenerry Crescent,14,[{'name': 'Furnis...,Abbotsford,302,2025-08-22 09:21:...,https://www.domai...
0.08416759,0.6407119,0.200222462,0.0748980343,Ray White Southbank,Eva Christodoulou,True,69,NULL,2,1,Brilliantly locat...,0.496447623,"Air conditioning,...",2025-08-27T14:48:...,DG04/18 Grosvenor...,https://rimh2.dom...,NULL,NULL,NULL,-37.8095856,recentlyUpdated,NULL,https://ww

In [15]:
# Load the shapefile (use the .shp file as entry point)
sf = gpd.read_file("../data/geo/shpfile/LOCALITY_POLYGON.shp")
sf = sf.to_crs(epsg=4326)  # Convert to lat/lon (WGS84)
sf = sf[["LOCALITY", "geometry"]]
sf["LOCALITY"] = sf["LOCALITY"].str.strip()  # Remove whitespace    
sf["suburb"] = sf["LOCALITY"].str.lower().str.strip()
sf.head()

,LOCALITY,geometry,suburb
0,MOLLONGGHIP,"POLYGON ((144.06544 -37.48382, 144.06438 -37.4...",mollongghip
1,NORTH BLACKWOOD,"POLYGON ((144.38037 -37.42376, 144.38126 -37.4...",north blackwood
2,BASALT,"POLYGON ((144.1184 -37.31148, 144.1184 -37.311...",basalt
3,LLANELLY,"POLYGON ((143.81737 -36.75048, 143.81679 -36.7...",llanelly
4,MURRABIT WEST,"POLYGON ((143.87075 -35.49319, 143.86939 -35.4...",murrabit west


In [16]:
# Function to get suburb name from lon/lat
def get_suburb(lon, lat, sf):
    pt = Point(lon, lat) # e.g., Point(144.9631, -37.8136)
    for _, row in sf.iterrows():
        poly = row['geometry']
        if poly.covers(pt):
            return row['suburb'] # return lower case suburb name
    return None

# try out the function
sub = get_suburb(144.9961565, -37.796893, sf)
print(sub)

abbotsford


In [17]:
school_locations_sf = spark.read.csv('../data/landing/schools/school_locations_2025.csv', header=True, inferSchema=True)
school_locations_sf.show(5)
#find number of distinct school types and entry count for each type
school_locations_sf.groupBy("School_Type").count().show()

+----------------+-----------+---------+--------------------+-----------+-------------+--------------------+--------------+--------------+-------------+----------------+---------------------+---------------------+--------------+------------+---------------+-------------+--------------------+--------------------+------+----------------+---------+----------+----------+
|Education_Sector|Entity_Type|School_No|         School_Name|School_Type|School_Status|      Address_Line_1|Address_Line_2|  Address_Town|Address_State|Address_Postcode|Postal_Address_Line_1|Postal_Address_Line_2|   Postal_Town|Postal_State|Postal_Postcode|Full_Phone_No|              Region|                Area|LGA_ID|        LGA_Name| LGA_TYPE|         X|         Y|
+----------------+-----------+---------+--------------------+-----------+-------------+--------------------+--------------+--------------+-------------+----------------+---------------------+---------------------+--------------+------------+---------------+---

In [18]:
def iter_rows(df, columns=None, stream=True):
    """
    Unified row iterator for Pandas/GeoPandas and PySpark DataFrames.
    Yields dicts. Optionally select a subset of `columns`.
    - stream=True uses toLocalIterator() for Spark (memory-safe).
    """
    mod = type(df).__module__

    # --- PySpark DataFrame ---
    if mod.startswith("pyspark.sql"):
        it = df.toLocalIterator() if stream else df.collect()
        for row in it:
            d = row.asDict(recursive=True)
            yield {k: d[k] for k in (columns or d.keys())}

    # --- Pandas / GeoPandas DataFrame ---
    else:
        cols = list(columns) if columns else list(df.columns)
        # itertuples is faster than iterrows; yields namedtuples
        for tup in df.itertuples(index=False, name=None):
            # name=None => plain tuples
            yield dict(zip(df.columns, tup)) if columns is None else \
                {c: v for c, v in zip(df.columns, tup) if c in cols}

In [19]:
# Function that finds the closest POI to a given rental listing
def find_closest_poi_sf(rental_lon, rental_lat, poi_df):
    rental_sub = get_suburb(rental_lon, rental_lat, sf)
    rental_point = Point(rental_lon, rental_lat)
    min_distance = float('inf') # Initialize with infinity
    closest_poi = None

    if rental_sub is None:
        return None, None  # Rental listing not in any suburb
    for poi in iter_rows(poi_df, columns=['School_Name', 'X', 'Y']):
        poi_sub = get_suburb(poi['X'], poi['Y'], sf)
        if poi_sub != rental_sub:
            continue  # Skip POIs not in the same suburb
        poi_point = Point(poi['X'], poi['Y'])
        distance = rental_point.distance(poi_point)
    
        if distance < min_distance:
            min_distance = distance
            closest_poi = poi
            
    return closest_poi, min_distance

# Example usage
rental_lon = 144.9961565
rental_lat = -37.796893
closest_poi, distance = find_closest_poi_sf(rental_lon, rental_lat, school_locations_sf)
if closest_poi:
    print(f"Closest POI: {closest_poi['School_Name']} at distance {distance}")

Closest POI: Sophia Mundi Steiner School at distance 0.011317426087662855


In [20]:
# Read in geojson file with geopandas
ptv_stops_gdf = gpd.read_file("../data/landing/ptv/public_transport_stops.geojson")
ptv_stops_gdf = ptv_stops_gdf.to_crs(epsg=4326)  # Convert to lat/lon (WGS84)

ptv_stops_gdf

,STOP_ID,STOP_NAME,MODE,geometry
0,17204,Wallan Station,REGIONAL TRAIN,POINT (145.00537 -37.41686)
1,19980,Melton Station,REGIONAL TRAIN,POINT (144.57222 -37.70336)
2,19981,Rockbank Station,REGIONAL TRAIN,POINT (144.65071 -37.72919)
3,19982,Deer Park Station,REGIONAL TRAIN,POINT (144.77083 -37.77727)
4,19998,Sunbury Station,REGIONAL TRAIN,POINT (144.72803 -37.57915)
...,...,...,...,...
29197,6586,Northcott St/Exford Rd,REGIONAL BUS,POINT (144.5743 -37.70666)
29198,6642,Northcott St/Exford Rd,REGIONAL BUS,POINT (144.57443 -37.70668)
29199,6644,Exford Rd/Staughton St,REGIONAL BUS,POINT (144.56982 -37.7024)
29200,7009,Harrison St/Marengo Crst,REGIONAL BUS,POINT (143.66323 -38.77712)


In [21]:
def find_closest_poi_gdf(rental_lon, rental_lat, poi_df):
    rental_sub = get_suburb(rental_lon, rental_lat, sf)
    rental_point = Point(rental_lon, rental_lat)
    min_distance = float('inf') # Initialize with infinity
    closest_poi = None

    if rental_sub is None:
        return None, None  # Rental listing not in any suburb
    for poi in iter_rows(poi_df, columns=['STOP_NAME', 'X', 'Y']):
        poi_sub = get_suburb(poi['X'], poi['Y'], sf)
        if poi_sub != rental_sub:
            continue  # Skip POIs not in the same suburb
        poi_point = Point(poi['X'], poi['Y'])
        distance = rental_point.distance(poi_point)
    
        if distance < min_distance: # returns the first closest POI
            min_distance = distance
            closest_poi = poi
            
    return closest_poi, min_distance

# Example usage
rental_lon = 144.9961565
rental_lat = -37.796893
closest_poi, distance = find_closest_poi_gdf(rental_lon, rental_lat, ptv_stops_sf)
if closest_poi:
    print(f"Closest POI: {closest_poi['STOP_NAME']} at distance {distance}")

NameError: name 'ptv_stops_sf' is not defined

In [ ]:
import openrouteservice as ors
import numpy as np
from sklearn.neighbors import BallTree
import requests
import time

client = ors.Client(key='eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6Ijk0YjdmMzczOGY0ZjQ3ZjY4YWRhZTZmN2FmYmQyODE4IiwiaCI6Im11cm11cjY0In0=')

rentals = sdf.select("full_address", "longitude", "latitude").limit(100).toPandas()
rentals= gpd.GeoDataFrame(
    rentals,
    geometry=[Point(xy) for xy in zip(rentals.longitude, rentals.latitude)],
    crs="EPSG:4326"
)

rentals

,full_address,longitude,latitude,geometry
0,"14 Federation Lane, Abbotsford VIC 3067",144.9961565,-37.796893,POINT (144.99616 -37.79689)
1,"106/609 Victoria Street, Abbotsford VIC 3067",145.0076834,-37.8110653,POINT (145.00768 -37.81107)
2,Abbotsford VIC 3067,145.0019064,-37.80210950000001,POINT (145.00191 -37.80211)
3,"4/2 Princes Street, Abbotsford VIC 3067",144.999856,-37.8092053,POINT (144.99986 -37.80921)
4,"45 Park St, Abbotsford VIC 3067",144.9939399,-37.8080424,POINT (144.99394 -37.80804)
...,...,...,...,...
95,"5/2-4 Watt St, Airport West VIC 3042",144.8799126,-37.7314592,POINT (144.87991 -37.73146)
96,"9 Thomas St, Airport West VIC 3042",144.876504,-37.7206421,POINT (144.8765 -37.72064)
97,"2/128 Bowes Avenue, Airport West VIC 3042",144.8863906,-37.7238332,POINT (144.88639 -37.72383)
98,"34A Walters Ave, Airport West VIC 3042",144.8864192,-37.73188030000001,POINT (144.88642 -37.73188)


In [ ]:
poi_rad = np.radians(np.c_[ptv_stops_gdf.geometry.y, ptv_stops_gdf.geometry.x])
tree = BallTree(poi_rad, metric="haversine")

def shortlist(point, k=6, max_km=2.0):
    pt = np.radians([[point.y, point.x]])
    dist, idx = tree.query(pt, k=k)
    dist_km = dist[0] * 6371.0088
    mask = dist_km <= max_km
    return ptv_stops_sf.iloc[idx[0][mask]]

# --- routing helper (OpenRouteService foot example) ---
API_KEY = "eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6Ijk0YjdmMzczOGY0ZjQ3ZjY4YWRhZTZmN2FmYmQyODE4IiwiaCI6Im11cm11cjY0In0="
BASE_URL = "https://api.openrouteservice.org/v2/matrix/driving-car"

def routed_distance(source, targets, max_attempts=5, base_wait=1.0):
    coords = [[source.x, source.y]] + [[pt.x, pt.y] for pt in targets]
    body = {"locations": coords, "sources": [0], "metrics": ["distance", "duration"]}
    for attempt in range(1, max_attempts + 1):
        resp = requests.post(BASE_URL, json=body,
                            headers={"Authorization": API_KEY},
                            timeout=30)

        if resp.status_code == 429:
            wait = base_wait * (2 ** (attempt - 1))  # 1, 2, 4, 8, ...
            time.sleep(wait)
            continue

        resp.raise_for_status()
        data = resp.json()
        return data["distances"][0][1:], data["durations"][0][1:]

    raise RuntimeError(f"ORS matrix still returning 429 after {max_attempts} retries")
# --- main loop ---
rows = []
for _, row in rentals.iterrows():
    candidates = shortlist(row.geometry, k=10, max_km=3)
    if candidates.empty:
        continue

    distances, durations = routed_distance(row.geometry, candidates.geometry, max_attempts=5, base_wait=1.0)
    idx = int(np.argmin(durations))  # use distance if you prefer metres
    chosen = candidates.iloc[idx]

    rows.append({
        "rental_id": row["full_address"],
        "poi_id": chosen["STOP_NAME"],
        "route_dist_m": distances[idx],
        "route_dur_s": durations[idx]
    })

In [ ]:
import pandas as pd
closest = pd.DataFrame(rows)
closest

,rental_id,poi_id,route_dist_m,route_dur_s
0,"14 Federation Lane, Abbotsford VIC 3067",Lulie St,307.36,51.77
1,"106/609 Victoria Street, Abbotsford VIC 3067",Leslie St/Victoria St #23,2.42,0.17
2,Abbotsford VIC 3067,Clarke St/Johnston St,189.53,33.30
3,"4/2 Princes Street, Abbotsford VIC 3067",Church St/Victoria St #21,281.79,37.71
4,"45 Park St, Abbotsford VIC 3067",Victoria St,293.38,56.78
...,...,...,...,...
95,"5/2-4 Watt St, Airport West VIC 3042",Grange Rd/Fullarton Rd,323.38,47.71
96,"9 Thomas St, Airport West VIC 3042",Thomas St/Etzel St,129.94,29.31
97,"2/128 Bowes Avenue, Airport West VIC 3042",King St/Matthews Ave,510.76,62.36
98,"34A Walters Ave, Airport West VIC 3042",Creswell Ave/Fullarton Rd,282.59,55.59


check if capped all 

In [ ]:
from geopy.distance import geodesic

lat1, lon1 = -37.796893, 144.9961565  # Melbourne CBD
dist = 5000                  # metres
tags = "theatre|cafe|nightclub|kindergarten|doctors|fuel|bank|library|cinema|restaurant|atm|bar|fast_food|pharmacy|veterinary|taxi|brothel|university|police|events_venue|college|car_rental|clinic|community_centre|courier|food_court|social_facility|parking_space|hospital|waste_disposal|parcel_locker|charging_station|coworking_space|meeting_point|motorcycle_parking|childcare|social_centre|music_venue|healthcare|waste_transfer_station|casino|fire_station|student_accommodation|retail|prison|nursing_home|events_centre|exhibition_centre|conference_centre|biergarten|bus_station"
query = f"""
[out:json][timeout:25];
nwr["amenity"~"{tags}"](around:{dist},{lat},{lon});
out center;
"""

url = "https://overpass-api.de/api/interpreter"
resp = requests.post(url, data=query)
resp.raise_for_status()
data = resp.json()

rows = []
for element in data["elements"]:
    
    props = element.get("tags", {})
    name = props.get("name", "Unnamed")
    lat = element.get("lat") or element["center"]["lat"]
    lon = element.get("lon") or element["center"]["lon"]
    distance_m = geodesic((lat1, lon1), (lat, lon)).meters
    rows.append({"name": name,
                "amenity": props.get("amenity"),
                "geometry": Point(lon, lat),
                "distance_m": distance_m})  # Calculate distance from the reference point

pois_gdf = gpd.GeoDataFrame(rows, crs="EPSG:4326")
pois_gdf.head()

,name,amenity,geometry,distance_m
0,Regent Theatre,theatre,POINT (144.9676 -37.8155),3254.048075
1,Books n Bites Cafe,cafe,POINT (144.95832 -37.80175),3376.004301
2,Bennetts Lane Jazz Club,nightclub,POINT (144.96764 -37.80898),2847.379288
3,Brunswick Crèche & Day Nursery,kindergarten,POINT (144.96629 -37.77225),3794.874500
4,Rubber Duck Cafe,cafe,POINT (144.99063 -37.78908),993.933704


In [ ]:
# Show counts of each amenity type and min distance
pois_gdf.groupby("amenity").agg(count=("name", "count"), min_distance_m=("distance_m", "min"))

,count,min_distance_m
amenity,,
atm,104,260.055533
bank,88,567.360620
bar,298,332.306051
biergarten,1,3604.761943
brothel,7,1322.296003
bus_station,1,4431.036387
cafe,937,342.862233
car_rental,22,1731.262016
casino,3,4414.967720


In [ ]:
set(pois_gdf['amenity'].unique()) - set(my_list)

{'internet_cafe', 'juice_bar', 'tool_library'}

In [ ]:
"|".join(list(pois_gdf["amenity"].unique()))

'theatre|cafe|nightclub|kindergarten|doctors|fuel|bank|library|cinema|restaurant|atm|bar|fast_food|pharmacy|veterinary|taxi|brothel|university|internet_cafe|police|events_venue|college|car_rental|clinic|community_centre|courier|food_court|social_facility|parking_space|hospital|waste_disposal|parcel_locker|charging_station|coworking_space|meeting_point|motorcycle_parking|childcare|juice_bar|social_centre|tool_library|music_venue|healthcare|waste_transfer_station|casino|fire_station|student_accommodation|retail|prison|nursing_home|events_centre|exhibition_centre|conference_centre|biergarten|bus_station'

In [ ]:
my_list = "theatre|cafe|nightclub|kindergarten|doctors|fuel|bank|library|cinema|restaurant|atm|bar|fast_food|pharmacy|veterinary|taxi|brothel|university|police|events_venue|college|car_rental|clinic|community_centre|courier|food_court|social_facility|parking_space|hospital|waste_disposal|parcel_locker|charging_station|coworking_space|meeting_point|motorcycle_parking|childcare|social_centre|music_venue|healthcare|waste_transfer_station|casino|fire_station|student_accommodation|retail|prison|nursing_home|events_centre|exhibition_centre|conference_centre|biergarten|bus_station".split("|")